
## Chapter 3: Schemas and Types - The Blueprint

The Schema is the heart of any GraphQL API. It acts as the contract between the client and the server, defining exactly what data can be queried and how it can be modified. We write schemas using the **Schema Definition Language (SDL)**, a simple and human-readable syntax.

In this chapter, we will dissect every component of the SDL with extensive code snippets and real-world examples.

### 3.1 The Root Types: Query, Mutation, and Subscription

Every GraphQL schema has three special "root" types that serve as entry points into the graph. Think of these as the front doors to your API.

```graphql
schema {
  query: Query
  mutation: Mutation
  subscription: Subscription
}
```

*   **Query:** The entry point for reading data (Read operations).
*   **Mutation:** The entry point for modifying data (Create, Update, Delete).
*   **Subscription:** The entry point for real-time data streams (Listen operations).

**Example: A Basic Root Structure**

```graphql
type Query {
  "Returns a specific user by ID"
  user(id: ID!): User
  "Returns a list of all products"
  products: [Product!]!
}

type Mutation {
  "Create a new product"
  createProduct(input: CreateProductInput!): Product!
}

type Subscription {
  "Real-time updates when a product is created"
  onProductCreated: Product
}
```

**Explanation:**
*   The `Query` type defines fields like `user` and `products`. These are the fields you can request at the root of your query document.
*   The `!` symbol indicates a field is **Non-Null** (guaranteed to never be `null`).

### 3.2 Object Types and Fields

Object Types are the building blocks of your graph. They represent a domain object (like a User, a Product, or an Order) and define what properties are available.

```graphql
type User {
  id: ID!
  username: String!
  email: String!
  age: Int
  isActive: Boolean!
}
```

**Breakdown:**
*   `type User`: Defines a new Object Type named `User`.
*   `id: ID!`: A field named `id` of type `ID` (a unique identifier).
*   `age: Int`: A field named `age` of type `Int`. Notice there is no `!`, meaning this field can be `null` (perhaps the user didn't provide their age).

### 3.3 Scalar Types: Built-in vs. Custom Scalars

Scalars are the leaf nodes of the query tree. They resolve to a concrete value (strings, numbers, booleans) rather than an object with fields.

**Built-in Scalars:**
GraphQL ships with five default scalars:

1.  `Int`: A signed 32-bit integer.
2.  `Float`: A signed double-precision floating-point value.
3.  `String`: A UTF-8 character sequence.
4.  `Boolean`: `true` or `false`.
5.  `ID`: A unique identifier, serialized as a String, but often used for caching keys.

**Custom Scalars:**
Sometimes built-in types aren't enough. You might need to enforce specific formats like Dates, Emails, or URLs.

```graphql
scalar Date
scalar Email

type Event {
  id: ID!
  title: String!
  date: Date!
  organizerEmail: Email!
}
```

**Implementation Note:**
Defining the scalar in the schema is only half the battle. In your server code (Chapter 7), you must define how to parse, serialize, and validate these values.

*   *Serialization:* Converting the internal value to JSON (e.g., `Date` object -> ISO String).
*   *Parsing:* Converting JSON input to internal value (e.g., ISO String -> `Date` object).

### 3.4 Enumeration Types (`Enum`)

Enums (Enumerations) are a special kind of scalar that is restricted to a particular set of allowed values. This allows you to:
1.  Validate that any argument of this type is one of the allowed values.
2.  Communicate through the type system that a field will always be one of a finite set of values.

```graphql
enum Role {
  ADMIN
  USER
  GUEST
}

enum ProductStatus {
  DRAFT
  PUBLISHED
  ARCHIVED
}

type Product {
  id: ID!
  name: String!
  status: ProductStatus!
}

type Query {
  "Filter products by status"
  products(status: ProductStatus): [Product!]!
}
```

**Usage Example:**
A client can now query:
```graphql
query GetDraftProducts {
  products(status: DRAFT) {
    name
  }
}
```
If they try `products(status: DELETED)`, the server will throw a validation error before execution even begins.

### 3.5 Lists and Non-Null Markers

This is often a source of confusion for beginners. The `!` (Non-Null) marker can be applied to the type itself or the list wrapper `[...]`.

Let's visualize the difference with `User` lists.

**Case 1: `[User]`**
*   The field can be `null`.
*   The list can contain `null` values.
*   Example: `null` or `[null, { id: "1" }]`

**Case 2: `[User!]`**
*   The field can be `null`.
*   The list **cannot** contain `null` values. Every item in the array must be a User object.
*   Example: `null` or `[{ id: "1" }, { id: "2" }]`

**Case 3: `[User]!`**
*   The field **cannot** be `null`. It will always return a list (even if empty).
*   The list **can** contain `null` values.
*   Example: `[]` or `[null, { id: "1" }]`

**Case 4: `[User!]!` (Recommended)**
*   The field cannot be `null`.
*   The list cannot contain `null` values.
*   Example: `[]` or `[{ id: "1" }, { id: "2" }]`

**Snippet:**
```graphql
type Query {
  users: [User!]!  # Guarantee: You get a list, and every item is a User.
  searchUsers(query: String!): [User] # Might return null, might have nulls inside.
}
```

### 3.6 Input Types: Structuring Mutation Arguments

When creating or updating data, you often need to pass complex objects as arguments. GraphQL allows you to define **Input Types** specifically for this purpose.

**Why not just use Object Types?**
Object Types can have fields that are Interfaces or Unions, or fields that take arguments. Input types are strictly for input data and cannot be used in output locations, ensuring a clear separation of concerns.

```graphql
input CreateProductInput {
  name: String!
  price: Float!
  category: ProductCategory!
  tags: [String!]!
}

enum ProductCategory {
  ELECTRONICS
  CLOTHING
  BOOKS
}

type Mutation {
  createProduct(input: CreateProductInput!): Product!
}
```

**Client Usage:**
```graphql
mutation CreateNewProduct {
  createProduct(input: {
    name: "GraphQL Handbook",
    price: 29.99,
    category: BOOKS,
    tags: ["programming", "api"]
  }) {
    id
    name
  }
}
```

### 3.7 Interfaces: Implementing Polymorphism

An Interface is an abstract type that includes a certain set of fields that other types must include to implement the interface. This allows you to query for fields that exist on multiple different types.

**Scenario:** You have `Book` and `Author`. Both can be "Searchable" and have a title/name.

```graphql
interface Searchable {
  id: ID!
  searchText: String!
}

type Book implements Searchable {
  id: ID!
  searchText: String! # Must include interface fields
  author: String!
  pageCount: Int!
}

type Author implements Searchable {
  id: ID!
  searchText: String! # Must include interface fields
  bio: String!
}

type Query {
  search(term: String!): [Searchable!]!
}
```

**Querying with Inline Fragments:**
Since the `search` query returns `Searchable`, you only have direct access to `id` and `searchText`. To access specific fields (like `pageCount`), you must use **Inline Fragments**:

```graphql
query SearchItems {
  search(term: "GraphQL") {
    id
    searchText
    ... on Book {
      pageCount
    }
    ... on Author {
      bio
    }
  }
}
```

### 3.8 Union Types: Handling Heterogeneous Data

While Interfaces are great for shared fields, sometimes you need to return completely different types from the same query. A **Union** is a type that represents an object that could be one of a list of types.

**Scenario:** A `SearchResult` could be a `User` or a `Post` (which share no common fields).

```graphql
union SearchResult = User | Post

type User {
  id: ID!
  username: String!
}

type Post {
  id: ID!
  title: String!
}

type Query {
  search(query: String!): [SearchResult!]!
}
```

**Client Query:**
You must use inline fragments to specify which fields you want for each possible type.

```graphql
query GlobalSearch {
  search(query: "Alice") {
    ... on User {
      username
    }
    ... on Post {
      title
    }
  }
}
```

### 3.9 Best Practices for Schema Design

1.  **Naming Conventions:**
    *   **Types:** PascalCase (`UserAccount`, `ProductCategory`).
    *   **Fields:** camelCase (`firstName`, `productList`).
    *   **Enums:** SCREAMING_SNAKE_CASE (`STATUS_ACTIVE`).
2.  **Granularity:** Design fields to be small and composable. Avoid creating a generic `data: JSON` field, as it defeats the purpose of a strongly typed schema.
3.  **Mutations:** Always name mutations as verbs (e.g., `createUser`, `deleteProduct`, `updateEmail`). The payload should return the modified object and optionally a success status or error.

---

### 🚀 Next Up: Chapter 4 - Querying Data - Read Operations

**Summary:** We have built the blueprint; now it is time to read the map. In Chapter 4, we move to the client side. We will learn how to construct complex queries, traverse relationships, use variables for dynamic data, and organize our queries with Fragments. This is where the data graph comes to life for the consumer.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../1. foundations_and_philosophy/2. thinking_in_graphs.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='4. querying_data_read_operations.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
